# Pincone Embeddings Retriever

Here we will develop a retriever that will perform a similarity search between our query and vectors within our vector database.

Pinecone holds all of your embeddings in the "cloud". In order to find content that will add context to our query's answer, we will search this database for documents that are similar to the query. 

A similarity search is performed finding the closest vectors to the query vector within the n-dimensional space.

First we need to initialize a connection with Pinecone.

In [3]:
from typing import List, Iterator
import pandas as pd
import numpy as np
import os
import wget
from ast import literal_eval

# Pinecone's client library 
import pinecone


In [ ]:
api_key = os.getenv("PINECONE_API_KEY")